# Exercice 15 - Introduction a Kafka

## Objectifs
- Comprendre les concepts fondamentaux de Kafka
- Decouvrir l'architecture Kafka
- Creer des topics
- Explorer l'interface Kafka UI

---

## 1. Qu'est-ce que Kafka ?

**Apache Kafka** est une plateforme de streaming distribue.

```
+------------------------------------------------------------------+
|                    ARCHITECTURE KAFKA                            |
+------------------------------------------------------------------+
|                                                                  |
|   PRODUCTEURS           KAFKA BROKER           CONSOMMATEURS     |
|                                                                  |
|  +-----------+      +----------------+      +-----------+        |
|  |           |      |                |      |           |        |
|  | App Web   |----->|    TOPIC       |----->| Spark     |        |
|  |           |      |    orders      |      |           |        |
|  +-----------+      |                |      +-----------+        |
|                     | +------------+ |                           |
|  +-----------+      | | Partition 0| |      +-----------+        |
|  |           |      | +------------+ |      |           |        |
|  | IoT       |----->| | Partition 1| |----->| Analytics |        |
|  | Devices   |      | +------------+ |      |           |        |
|  +-----------+      | | Partition 2| |      +-----------+        |
|                     | +------------+ |                           |
|  +-----------+      |                |      +-----------+        |
|  |           |      +----------------+      |           |        |
|  | Script    |----->                  ----->| Dashboard |        |
|  | Python    |                              |           |        |
|  +-----------+                              +-----------+        |
|                                                                  |
+------------------------------------------------------------------+

Concepts cles :
- Topic     : categorie de messages (comme une table)
- Partition : sous-division d'un topic (parallelisme)
- Producer  : envoie des messages
- Consumer  : recoit des messages
- Broker    : serveur Kafka
- Offset    : position d'un message dans une partition
```

## 2. Cas d'usage de Kafka

```
+------------------------------------------------------------------+
|                    CAS D'USAGE KAFKA                             |
+------------------------------------------------------------------+
|                                                                  |
|  1. STREAMING DE DONNEES                                         |
|     - Logs en temps reel                                         |
|     - Metriques systeme                                          |
|     - Evenements utilisateur                                     |
|                                                                  |
|  2. INTEGRATION DE SYSTEMES                                      |
|     - Synchronisation de bases                                   |
|     - Communication microservices                                |
|     - ETL en temps reel                                          |
|                                                                  |
|  3. EVENT SOURCING                                               |
|     - Historique des evenements                                  |
|     - Audit trail                                                |
|     - Replay de donnees                                          |
|                                                                  |
|  4. ANALYTICS TEMPS REEL                                         |
|     - Dashboards en direct                                       |
|     - Alerting                                                   |
|     - Detection d'anomalies                                      |
|                                                                  |
+------------------------------------------------------------------+
```

## 3. Notre infrastructure Kafka

```
+------------------------------------------------------------------+
|                    NOTRE STACK                                   |
+------------------------------------------------------------------+
|                                                                  |
|  +----------------+          +-------------------+               |
|  |                |          |                   |               |
|  |  Kafka Broker  |<-------->|    Kafka UI       |               |
|  |  (port 9092)   |          |  (port 7080)      |               |
|  |                |          |                   |               |
|  +----------------+          +-------------------+               |
|         |                                                        |
|         |                                                        |
|         v                                                        |
|  +----------------+          +-------------------+               |
|  |                |          |                   |               |
|  | JupyterLab     |          |    Spark          |               |
|  | (Producer)     |          |  (Consumer)       |               |
|  |                |          |                   |               |
|  +----------------+          +-------------------+               |
|                                                                  |
+------------------------------------------------------------------+

URLs :
- Kafka Broker : broker:9092 (interne Docker)
- Kafka UI    : http://localhost:7080
```

## 4. Installation de la bibliotheque Kafka Python

In [1]:
# Installer kafka-python
!pip install kafka-python -q

print("kafka-python installe")

kafka-python installe


In [2]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer
from kafka.admin import NewTopic

# Configuration Kafka
KAFKA_BROKER = "broker:9092"

print(f"Configuration Kafka : {KAFKA_BROKER}")

Configuration Kafka : broker:9092


## 5. Tester la connexion a Kafka

In [3]:
# Tester la connexion
try:
    admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BROKER,
        client_id='test-connexion'
    )
    
    # Lister les topics existants
    topics = admin.list_topics()
    print("Connexion Kafka OK")
    print(f"Topics existants : {topics}")
    
except Exception as e:
    print(f"Erreur de connexion : {e}")

Erreur de connexion : NodeNotReadyError: 1


## 6. Creer un topic

In [7]:
import time
from kafka import KafkaAdminClient
from kafka.errors import NoBrokersAvailable, NodeNotReadyError

KAFKA_BROKER = "broker:9092"

def attendre_kafka(timeout=60):
    print("⏳ Tentative de connexion à Kafka...")
    start = time.time()
    
    while time.time() - start < timeout:
        try:
            # On tente juste de lister les topics pour voir si le broker répond
            admin = KafkaAdminClient(bootstrap_servers=KAFKA_BROKER)
            topics = admin.list_topics()
            admin.close()
            print("✅ Kafka est prêt !")
            return True
        except (NoBrokersAvailable, NodeNotReadyError):
            print(".", end="", flush=True) # Affiche un point à chaque échec
            time.sleep(2)
        except Exception as e:
            print(f"\nErreur inattendue : {e}")
            time.sleep(2)
            
    print("\n❌ Impossible de se connecter à Kafka après 60 secondes.")
    return False

# Si Kafka est prêt, on lance la création
if attendre_kafka():
    # Votre fonction de création de topic
    creer_topic("test-topic", partitions=3)
    creer_topic("commandes", partitions=3)
    creer_topic("logs", partitions=1)
    
    # Vérification finale
    print("\nTopics existants :")
    admin = KafkaAdminClient(bootstrap_servers=KAFKA_BROKER)
    print(admin.list_topics())
    admin.close()

⏳ Tentative de connexion à Kafka...
...............
❌ Impossible de se connecter à Kafka après 60 secondes.


In [5]:
# Creer des topics pour nos exercices
creer_topic("test-topic", partitions=3)
creer_topic("commandes", partitions=3)
creer_topic("logs", partitions=1)

Erreur lors de la création du topic : NodeNotReadyError: 1
Erreur lors de la création du topic : NodeNotReadyError: 1
Erreur lors de la création du topic : NodeNotReadyError: 1


In [6]:
# Lister les topics
admin = KafkaAdminClient(bootstrap_servers=KAFKA_BROKER)
topics = admin.list_topics()

print("Topics Kafka :")
print("=" * 30)
for topic in sorted(topics):
    if not topic.startswith("_"):  # Ignorer les topics internes
        print(f"  - {topic}")

admin.close()

NodeNotReadyError: NodeNotReadyError: 1

## 7. Envoyer des messages simples (Producer)

In [ ]:
# Creer un producer simple
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER,
    value_serializer=lambda v: v.encode('utf-8')  # Encoder en bytes
)

print("Producer cree")

In [ ]:
# Envoyer des messages
for i in range(5):
    message = f"Message numero {i+1}"
    future = producer.send("test-topic", value=message)
    
    # Attendre la confirmation
    result = future.get(timeout=10)
    print(f"Envoye: {message} -> Partition {result.partition}, Offset {result.offset}")

producer.flush()  # S'assurer que tout est envoye
print("\nTous les messages envoyes")

In [ ]:
# Fermer le producer
producer.close()

## 8. Lire des messages (Consumer)

In [ ]:
# Creer un consumer
consumer = KafkaConsumer(
    "test-topic",
    bootstrap_servers=KAFKA_BROKER,
    auto_offset_reset='earliest',  # Lire depuis le debut
    enable_auto_commit=True,
    group_id='test-group',
    value_deserializer=lambda v: v.decode('utf-8'),
    consumer_timeout_ms=5000  # Timeout apres 5 secondes sans message
)

print("Consumer cree, lecture des messages...")
print("=" * 50)

In [ ]:
# Lire les messages
messages_recus = 0

for message in consumer:
    print(f"Topic: {message.topic}")
    print(f"Partition: {message.partition}")
    print(f"Offset: {message.offset}")
    print(f"Valeur: {message.value}")
    print("-" * 30)
    messages_recus += 1

print(f"\nTotal messages recus: {messages_recus}")
consumer.close()

## 9. Interface Kafka UI

Kafka UI est accessible a l'adresse : **http://localhost:7080**

```
+------------------------------------------------------------------+
|                    KAFKA UI                                      |
+------------------------------------------------------------------+
|                                                                  |
|  Fonctionnalites :                                               |
|                                                                  |
|  1. TOPICS                                                       |
|     - Liste des topics                                           |
|     - Creation/suppression                                       |
|     - Configuration                                              |
|                                                                  |
|  2. MESSAGES                                                     |
|     - Visualisation des messages                                 |
|     - Filtrage par offset                                        |
|     - Envoi de messages                                          |
|                                                                  |
|  3. CONSUMERS                                                    |
|     - Groupes de consommateurs                                   |
|     - Lag (retard)                                               |
|     - Reset offset                                               |
|                                                                  |
|  4. BROKERS                                                      |
|     - Etat des brokers                                           |
|     - Metriques                                                  |
|                                                                  |
+------------------------------------------------------------------+
```

## 10. Supprimer un topic

In [ ]:
def supprimer_topic(nom_topic):
    """
    Supprime un topic Kafka.
    """
    try:
        admin = KafkaAdminClient(
            bootstrap_servers=KAFKA_BROKER,
            client_id='admin-delete'
        )
        
        admin.delete_topics([nom_topic])
        print(f"Topic '{nom_topic}' supprime")
        
    except Exception as e:
        print(f"Erreur : {e}")
    finally:
        admin.close()

# Exemple (commenté pour ne pas supprimer)
# supprimer_topic("test-topic")

---

## Exercice

**Objectif** : Experimenter avec Kafka

**Consigne** :
1. Creez un topic "exercice-topic" avec 2 partitions
2. Envoyez 10 messages personnalises
3. Lisez tous les messages
4. Verifiez dans Kafka UI (http://localhost:7080)

A vous de jouer :

In [ ]:
# TODO: Creer le topic
from kafka import KafkaAdminClient
from kafka.admin import NewTopic

KAFKA_BROKER = "broker:9092"

def create_exo_topic():
    admin = KafkaAdminClient(bootstrap_servers=KAFKA_BROKER)
    try:
        # Vérification pour ne pas créer de doublon
        if "exercice-topic" not in admin.list_topics():
            topic = NewTopic(name="exercice-topic", num_partitions=2, replication_factor=1)
            admin.create_topics([topic])
            print("Topic 'exercice-topic' créé avec succès (2 partitions).")
        else:
            print("Le topic 'exercice-topic' existe déjà.")
    except Exception as e:
        print(f"Erreur : {e}")
    finally:
        admin.close()

create_exo_topic()

In [ ]:
# TODO: Envoyer des messages
from kafka import KafkaProducer
import time

producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER,
    value_serializer=lambda v: v.encode('utf-8')
)

print("Début de l'envoi...")

for i in range(10):
    msg = f"Message Exercice #{i+1}"
    producer.send("exercice-topic", value=msg)
    print(f"Envoi : {msg}")
    time.sleep(0.2)

producer.flush() #pour forcer l'envoi des msg en mémoire
producer.close()
print("Tous les messages sont envoyés.")

In [ ]:
# TODO: Lire les messages
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    "exercice-topic",
    bootstrap_servers=KAFKA_BROKER,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='groupe-exercice',
    value_deserializer=lambda v: v.decode('utf-8'),
    consumer_timeout_ms=5000
)

print("Lecture des messages du topic 'exercice-topic' :")
print("-" * 50)

msg_count = 0
for msg in consumer:
    # Affichage de la partition
    print(f"Partition {msg.partition} | Offset {msg.offset} | Contenu : {msg.value}")
    msg_count += 1

print("-" * 50)
print(f"Terminé. {msg_count} messages reçus.")
consumer.close()

---

## Resume

Dans ce notebook, vous avez appris :
- Les **concepts fondamentaux** de Kafka (topic, partition, offset)
- Les **cas d'usage** typiques
- Comment **creer des topics**
- Comment **envoyer des messages** (Producer)
- Comment **lire des messages** (Consumer)
- Comment utiliser **Kafka UI**

### Prochaine etape
Dans le prochain notebook, nous approfondirons la creation de Producer Kafka avec Python.